# Runtime-Optimized VM Allocation for Benchmarks

This notebook implements a bin packing algorithm to allocate benchmarks to VMs based on actual HiGHS v1.10 runtime data, aiming to minimize total runtime variance across VMs.

## Important: HiGHS Variant Multiplier

**We run 5 HiGHS solver variants for each benchmark:**
1. `highs` (v1.10.0 standard)
2. `highs-hipo-ipm` (v1.11.0 with IPM solver)
3. `highs-hipo` (v1.11.0 with HiPO solver + 128 block size)
4. `highs-hipo-32` (v1.11.0 with HiPO + 32 block size)
5. `highs-hipo-64` (v1.11.0 with HiPO + 64 block size)

Therefore, **actual VM runtime = base HiGHS runtime × 5**

This multiplier is applied to all runtime calculations and VM allocations to accurately reflect the actual wall-clock time each VM will take to complete all benchmark variants.

In [49]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import heapq
from collections import defaultdict

# CONFIGURATION
# MAX_RUNTIME_PER_VM_SECONDS = 3600  # Set to a number (e.g., 3600 for 1 hour) to cap VM runtime, or None for no limit
MAX_RUNTIME_PER_VM_SECONDS = None  # Set to a number (e.g., 3600 for 1 hour) to cap VM runtime, or None for no limit
hipo_variants = ['highs-hipo-ipm', 'highs-hipo-128', 'highs-hipo-32', 'highs-hipo-64']
NUM_HIGHS_VARIANTS = len(hipo_variants) + 1  # +1 for standard 'highs'

## Load and Process Runtime Data

In [50]:
# Load HiGHS runtime data (including HiGHS-hipo if available)
highs_data = pd.read_csv('../main_results.csv', header=None, names=[
    'Benchmark', 'Size', 'Solver', 'Solver Version', 'Solver Release Year',
    'Status', 'Termination Condition', 'Runtime (s)', 'Memory Usage (MB)',
    'Objective Value', 'Max Integrality Violation', 'Duality Gap',
    'Reported Runtime (s)', 'Timeout', 'Hostname', 'Run ID', 'Timestamp'
])

# Filter for HiGHS 1.10.0 (2025) and HiGHS-hipo successful runs
highs_v110 = highs_data[
    ((highs_data['Solver Version'] == '1.10.0') |
     (highs_data['Solver'] == 'highs')) &
    (highs_data['Status'] == 'ok')
]


# Filter runs with runtime between 15 minutes (900s) and 1 hour (3600s)
# MIN_RUNTIME_SECONDS = 900   # 15 minutes
# MAX_RUNTIME_SECONDS = 3600  # 1 hour

# highs_v110 = highs_v110[
#     (highs_v110['Runtime (s)'].astype(float) >= MIN_RUNTIME_SECONDS) &
#     (highs_v110['Runtime (s)'].astype(float) <= MAX_RUNTIME_SECONDS)
# ]

print(f"Found {len(highs_v110)} successful HiGHS v1.10/hipo benchmark runs")
# print(f"Runtime range: {MIN_RUNTIME_SECONDS}s ({MIN_RUNTIME_SECONDS/60:.0f} min) to {MAX_RUNTIME_SECONDS}s ({MAX_RUNTIME_SECONDS/3600:.0f} hour)")
print(f"Solvers included: {highs_v110['Solver'].unique()}")
highs_v110.head()

Found 286 successful HiGHS v1.10/hipo benchmark runs
Solvers included: ['highs']


,Benchmark,Size,Solver,Solver Version,Solver Release Year,Status,Termination Condition,Runtime (s),Memory Usage (MB),Objective Value,Max Integrality Violation,Duality Gap,Reported Runtime (s),Timeout,Hostname,Run ID,Timestamp
8,genx-6_three_zones_w_multistage-no_uc,3-1h,highs,1.5.0.dev0,2022,ok,optimal,176.0468776710004,508.196,16995.00181074802,NaN,NaN,175.01348757743835,3600.0,benchmark-instance-z-m37,20250429_090606_benchmark-instance-z-m37,2025-04-29 13:40:49.151616
11,pypsa-eur-elec-trex,6-12h,highs,1.5.0.dev0,2022,ok,optimal,129.05456772100297,675.904,7243765719.563399,NaN,NaN,127.46630811691284,3600.0,benchmark-instance-z-m37,20250429_090606_benchmark-instance-z-m37,2025-04-29 14:08:04.724272
14,times-etimeseu-france-elec+heat-co2-multi_stage,1-64ts,highs,1.5.0.dev0,2022,ok,optimal,21.899618912000733,351.004,427842.1201974797,NaN,NaN,21.036941528320312,3600.0,benchmark-instance-z-m37,20250429_090606_benchmark-instance-z-m37,2025-04-29 14:15:49.989735
21,genx-6_three_zones_w_multistage-no_uc,3-1h,highs,1.6.0.dev0,2023,ok,optimal,167.53820870500203,515.24,16995.00181074802,NaN,NaN,166.69502234458923,3600.0,benchmark-instance-z-m37,20250429_090606_benchmark-instance-z-m37,2025-04-29 18:34:28.920670
25,pypsa-eur-elec-trex,6-12h,highs,1.6.0.dev0,2023,ok,optimal,125.93406565899932,667.8,7243765719.563399,NaN,NaN,124.6388909816742,3600.0,benchmark-instance-z-m37,20250429_090606_benchmark-instance-z-m37,2025-04-29 19:04:02.616776


In [51]:
# Create benchmark runtime mapping
# IMPORTANT: We multiply runtime by NUM_HIGHS_VARIANTS since each benchmark
# is run with 5 different HiGHS variants (highs, highs-hipo-128, highs-hipo-ipm, highs-hipo-32, highs-hipo-64)
benchmark_runtimes = {}
for _, row in highs_v110.iterrows():
    benchmark_key = f"{row['Benchmark']}-{row['Size']}"
    try:
        base_runtime = float(row['Runtime (s)'])
        # Multiply by number of variants since we run all variants for each benchmark
        actual_vm_runtime = base_runtime * NUM_HIGHS_VARIANTS
        benchmark_runtimes[benchmark_key] = actual_vm_runtime
    except Exception as e:
        print(f"Error processing row: {row}")
        print(f"Error: {e}")

print(f"Runtime data available for {len(benchmark_runtimes)} benchmarks")
print(f"Base total runtime (single variant): {sum(benchmark_runtimes.values())/NUM_HIGHS_VARIANTS} seconds ({sum(benchmark_runtimes.values())/NUM_HIGHS_VARIANTS/3600:.1f} hours)")
print(f"Actual total runtime ({NUM_HIGHS_VARIANTS} variants): {sum(benchmark_runtimes.values())} seconds ({sum(benchmark_runtimes.values())/3600:.1f} hours)")
print(f"Multiplier applied: {NUM_HIGHS_VARIANTS}x (running {NUM_HIGHS_VARIANTS} HiGHS variants per benchmark)")

Runtime data available for 78 benchmarks
Base total runtime (single variant): 120329.967571507 seconds (33.4 hours)
Actual total runtime (5 variants): 601649.837857535 seconds (167.1 hours)
Multiplier applied: 5x (running 5 HiGHS variants per benchmark)


## Load Benchmark Metadata

In [52]:
# Load benchmark metadata to get size categories and URLs
meta = yaml.safe_load(open('../results/metadata.yaml'))

# Create a lookup for metadata
metadata_lookup = {}
for name, benchmark in meta['benchmarks'].items():
    for size_info in benchmark['Sizes']:
        instance_key = f"{name}-{size_info['Name']}"
        metadata_lookup[instance_key] = size_info

# Categorize benchmarks by size and create instances list
# Use ONLY the filtered highs_v110 dataset
benchmarks_by_size = {'S': [], 'M': [], 'L': []}
all_benchmark_instances = []

for _, row in highs_v110.iterrows():
    instance_key = f"{row['Benchmark']}-{row['Size']}"

    # Get metadata for this instance
    size_info = metadata_lookup.get(instance_key)
    if size_info is None:
        print(f"Warning: No metadata found for {instance_key}")
        continue

    # Apply variant multiplier to runtime
    base_runtime = float(row['Runtime (s)'])
    actual_runtime = base_runtime * NUM_HIGHS_VARIANTS

    instance = {
        'name': row['Benchmark'],
        'size_name': row['Size'],
        'size_category': size_info['Size'],
        'instance_key': instance_key,
        'runtime': actual_runtime,  # Runtime multiplied by number of variants
        'base_runtime': base_runtime,  # Store original single-variant runtime for reference
        'num_variables': size_info.get('Num. variables', 0),
        'num_constraints': size_info.get('Num. constraints', 0),
        'url': size_info['URL']
    }

    benchmarks_by_size[size_info['Size']].append(instance)
    all_benchmark_instances.append(instance)

print(f"Total benchmark instances (from filtered dataset): {len(all_benchmark_instances)}")
for size, instances in benchmarks_by_size.items():
    print(f"  {size}: {len(instances)}")
print(f"All instances have runtime data from highs_v110")
print(f"Runtime multiplier: {NUM_HIGHS_VARIANTS}x (to account for running all HiGHS variants)")

Total benchmark instances (from filtered dataset): 286
  S: 60
  M: 221
  L: 5
All instances have runtime data from highs_v110
Runtime multiplier: 5x (to account for running all HiGHS variants)


## Bin Packing Algorithms

In [53]:
class VMAllocation:
    def __init__(self, vm_id: int):
        self.vm_id = vm_id
        self.benchmarks = []
        self.total_runtime = 0.0

    def add_benchmark(self, benchmark: dict):
        """Add benchmark with real runtime data only"""
        if benchmark['runtime'] is None:
            raise ValueError(f"Benchmark {benchmark['instance_key']} has no runtime data!")

        self.benchmarks.append(benchmark)
        self.total_runtime += benchmark['runtime']

    def get_total_runtime(self):
        return self.total_runtime

    def __lt__(self, other):
        # For heap operations - compare by total runtime
        return self.total_runtime < other.total_runtime

In [54]:
def first_fit_decreasing(benchmarks: list[dict], num_vms: int) -> list[VMAllocation]:
    """
    First Fit Decreasing bin packing algorithm.
    Uses ONLY benchmarks with real runtime data.
    """
    # Filter to only benchmarks with real runtime data
    runtime_benchmarks = [b for b in benchmarks if b['runtime'] is not None]
    print(f"Using {len(runtime_benchmarks)} benchmarks with real runtime data (filtered from {len(benchmarks)} total)")

    # Create VMs
    vms = [VMAllocation(i) for i in range(num_vms)]

    # Sort benchmarks by runtime (descending)
    sorted_benchmarks = sorted(runtime_benchmarks, key=lambda x: x['runtime'], reverse=True)

    # Assign benchmarks to VMs
    for benchmark in sorted_benchmarks:
        # Find VM with minimum current runtime
        min_vm = min(vms, key=lambda vm: vm.total_runtime)
        min_vm.add_benchmark(benchmark)

    return vms

In [55]:
def longest_processing_time_first(benchmarks: list[dict], num_vms: int) -> list[VMAllocation]:
    """
    Longest Processing Time First algorithm using a min-heap.
    Uses ONLY benchmarks with real runtime data.
    """
    # Filter to only benchmarks with real runtime data
    runtime_benchmarks = [b for b in benchmarks if b['runtime'] is not None]
    print(f"Using {len(runtime_benchmarks)} benchmarks with real runtime data (filtered from {len(benchmarks)} total)")

    # Create VMs and initialize heap
    vms = [VMAllocation(i) for i in range(num_vms)]
    vm_heap = list(vms)  # Min-heap based on total runtime
    heapq.heapify(vm_heap)

    # Sort benchmarks by runtime (descending)
    sorted_benchmarks = sorted(runtime_benchmarks, key=lambda x: x['runtime'], reverse=True)

    # Assign benchmarks
    for benchmark in sorted_benchmarks:
        # Get VM with minimum load
        min_vm = heapq.heappop(vm_heap)
        min_vm.add_benchmark(benchmark)
        # Re-insert VM into heap
        heapq.heappush(vm_heap, min_vm)

    return vms

In [56]:
def balanced_partition(benchmarks: list[dict], num_vms: int, max_runtime_per_vm: float = None) -> list[VMAllocation]:
    """
    Balanced partition algorithm that tries to achieve equal total runtime per VM.
    Uses ONLY benchmarks with real runtime data.

    Args:
        benchmarks: List of benchmark dictionaries with runtime data
        num_vms: Initial number of VMs to create
        max_runtime_per_vm: Maximum runtime allowed per VM (in seconds). If None, no limit.
    """
    # Filter to only benchmarks with real runtime data
    runtime_benchmarks = [b for b in benchmarks if b['runtime'] is not None]
    print(f"Using {len(runtime_benchmarks)} benchmarks with real runtime data (filtered from {len(benchmarks)} total)")

    # Calculate total runtime and target per VM
    total_runtime = sum(b['runtime'] for b in runtime_benchmarks)
    target_runtime_per_vm = total_runtime / num_vms

    # If max_runtime_per_vm is set and target exceeds it, increase num_vms
    if max_runtime_per_vm is not None and target_runtime_per_vm > max_runtime_per_vm:
        original_num_vms = num_vms
        num_vms = int(np.ceil(total_runtime / max_runtime_per_vm))
        target_runtime_per_vm = total_runtime / num_vms
        print(f"⚠️  Target runtime {target_runtime_per_vm/3600:.1f}h exceeds max {max_runtime_per_vm/3600:.1f}h")
        print(f"   Increasing VMs from {original_num_vms} to {num_vms} to respect runtime cap")

    print(f"Target runtime per VM: {target_runtime_per_vm:.1f} seconds ({target_runtime_per_vm/3600:.1f} hours)")
    if max_runtime_per_vm is not None:
        print(f"Max runtime per VM: {max_runtime_per_vm:.1f} seconds ({max_runtime_per_vm/3600:.1f} hours)")

    # Create VMs
    vms = [VMAllocation(i) for i in range(num_vms)]

    # Sort benchmarks by runtime (descending)
    sorted_benchmarks = sorted(runtime_benchmarks, key=lambda x: x['runtime'], reverse=True)

    # Assign benchmarks with balance consideration
    for benchmark in sorted_benchmarks:
        benchmark_runtime = benchmark['runtime']

        # Find VM that would be closest to target after adding this benchmark
        best_vm = None
        best_score = float('inf')

        for vm in vms:
            current_runtime = vm.total_runtime
            after_runtime = current_runtime + benchmark_runtime

            # Skip if this would exceed max runtime (only if max is set)
            if max_runtime_per_vm is not None and after_runtime > max_runtime_per_vm:
                # Check if any VM can still fit this benchmark
                if current_runtime + benchmark_runtime <= max_runtime_per_vm * 1.05:  # Allow 5% overflow
                    pass  # Continue to consider this VM
                else:
                    continue  # Skip this VM

            # Score based on deviation from target
            score = abs(after_runtime - target_runtime_per_vm)

            # Prefer VMs that are under-loaded
            if current_runtime < target_runtime_per_vm:
                score *= 0.8  # Bonus for under-loaded VMs

            if score < best_score:
                best_score = score
                best_vm = vm

        # Add benchmark to best VM (or create new VM if needed)
        if best_vm is None and max_runtime_per_vm is not None:
            # All VMs are at capacity, create a new one
            print(f"⚠️  All VMs at capacity, creating additional VM for benchmark {benchmark['instance_key']}")
            best_vm = VMAllocation(len(vms))
            vms.append(best_vm)

        if best_vm is not None:
            best_vm.add_benchmark(benchmark)
        else:
            print(f"❌ Could not allocate benchmark {benchmark['instance_key']} (runtime: {benchmark_runtime:.1f}s)")

    return vms

## Algorithm Comparison

In [57]:
def analyze_allocation(vms: list[VMAllocation], algorithm_name: str):
    """
    Analyze and print statistics for a VM allocation.
    """
    runtimes = [vm.total_runtime for vm in vms]

    # Filter out empty VMs (should not happen with real runtime data only)
    active_vms = [vm for vm in vms if vm.total_runtime > 0]
    active_runtimes = [vm.total_runtime for vm in active_vms]

    print(f"\n=== {algorithm_name} ===")
    print(f"Total VMs created: {len(vms)}")
    print(f"Active VMs (with benchmarks): {len(active_vms)}")
    print(f"Empty VMs: {len(vms) - len(active_vms)}")

    if len(active_vms) > 0:
        print(f"Total runtime: {sum(active_runtimes):.1f} seconds ({sum(active_runtimes)/3600:.1f} hours)")
        print(f"Average runtime per active VM: {np.mean(active_runtimes):.1f} seconds ({np.mean(active_runtimes)/3600:.1f} hours)")
        print(f"Runtime standard deviation: {np.std(active_runtimes):.1f} seconds ({np.std(active_runtimes)/3600:.1f} hours)")
        print(f"Min runtime: {min(active_runtimes):.1f} seconds ({min(active_runtimes)/3600:.1f} hours)")
        print(f"Max runtime: {max(active_runtimes):.1f} seconds ({max(active_runtimes)/3600:.1f} hours)")
        print(f"Runtime ratio (max/min): {max(active_runtimes)/min(active_runtimes):.2f}")

        # Efficiency (how balanced the allocation is)
        efficiency = 1 - (np.std(active_runtimes) / np.mean(active_runtimes))
        print(f"Load balance efficiency: {efficiency:.3f} (1.0 = perfect balance)")
    else:
        print("No active VMs found!")
        efficiency = 0

    return {
        'algorithm': algorithm_name,
        'total_runtime': sum(active_runtimes) if active_vms else 0,
        'std_runtime': np.std(active_runtimes) if active_vms else 0,
        'max_runtime': max(active_runtimes) if active_vms else 0,
        'min_runtime': min(active_runtimes) if active_vms else 0,
        'efficiency': efficiency,
        'runtimes': runtimes,
        'active_vms': len(active_vms),
        'num_vms': len(vms)
    }

In [58]:
# Use ONLY benchmarks that have real runtime data - no estimation!
benchmarks_with_runtime = [b for b in all_benchmark_instances if b['runtime'] is not None]
print(f"Using {len(benchmarks_with_runtime)} benchmarks with real HiGHS runtime data")
print(f"Excluded {len(all_benchmark_instances) - len(benchmarks_with_runtime)} benchmarks without runtime data")
print(f"Base total runtime (single HiGHS variant): {sum(b['base_runtime'] for b in benchmarks_with_runtime)/3600:.1f} hours")
print(f"Actual total runtime ({NUM_HIGHS_VARIANTS} HiGHS variants): {sum(b['runtime'] for b in benchmarks_with_runtime)/3600:.1f} hours")
print(f"Runtime multiplier: {NUM_HIGHS_VARIANTS}x")

if MAX_RUNTIME_PER_VM_SECONDS is not None:
    print(f"\n⚙️  Runtime cap enabled: {MAX_RUNTIME_PER_VM_SECONDS} seconds ({MAX_RUNTIME_PER_VM_SECONDS/3600:.1f} hours) per VM")
else:
    print(f"\n⚙️  No runtime cap configured (unlimited)")

# Separate L-size benchmarks for highmem machines
l_size_benchmarks = [b for b in benchmarks_with_runtime if b['size_category'] == 'L']
non_l_benchmarks = [b for b in benchmarks_with_runtime if b['size_category'] != 'L']

print(f"\nBenchmark separation by size category:")
print(f"  L-size (highmem): {len(l_size_benchmarks)} benchmarks, {sum(b['runtime'] for b in l_size_benchmarks)/3600:.1f} hours (with {NUM_HIGHS_VARIANTS} variants)")
print(f"  S/M-size (standard): {len(non_l_benchmarks)} benchmarks, {sum(b['runtime'] for b in non_l_benchmarks)/3600:.1f} hours (with {NUM_HIGHS_VARIANTS} variants)")

# Test different numbers of VMs for each category
results = []

# L-size benchmarks (fewer VMs since they need highmem)
print(f"\n{'='*50}")
print(f"TESTING L-SIZE BENCHMARKS (HIGHMEM MACHINES)")
print(f"{'='*50}")

l_vm_options = [2, 3, 4, 5] if len(l_size_benchmarks) > 0 else [1]
for num_vms in l_vm_options:
    if len(l_size_benchmarks) == 0:
        print("No L-size benchmarks with runtime data")
        break

    print(f"\nTesting {num_vms} highmem VMs for L-size benchmarks:")

    bp_vms = balanced_partition(l_size_benchmarks, num_vms, MAX_RUNTIME_PER_VM_SECONDS)
    bp_result = analyze_allocation(bp_vms, f"L-size Balanced Partition ({num_vms} VMs)")
    bp_result['num_vms'] = num_vms
    bp_result['size_category'] = 'L'
    results.append(bp_result)

# S/M-size benchmarks (more VMs with standard machines)
print(f"\n{'='*50}")
print(f"TESTING S/M-SIZE BENCHMARKS (STANDARD MACHINES)")
print(f"{'='*50}")

sm_vm_options = [8, 10, 12, 15]
for num_vms in sm_vm_options:
    if len(non_l_benchmarks) == 0:
        print("No S/M-size benchmarks with runtime data")
        break

    print(f"\nTesting {num_vms} standard VMs for S/M-size benchmarks:")

    bp_vms = balanced_partition(non_l_benchmarks, num_vms, MAX_RUNTIME_PER_VM_SECONDS)
    bp_result = analyze_allocation(bp_vms, f"S/M-size Balanced Partition ({num_vms} VMs)")
    bp_result['num_vms'] = num_vms
    bp_result['size_category'] = 'S/M'
    results.append(bp_result)

Using 286 benchmarks with real HiGHS runtime data
Excluded 0 benchmarks without runtime data
Base total runtime (single HiGHS variant): 61.9 hours
Actual total runtime (5 HiGHS variants): 309.6 hours
Runtime multiplier: 5x

⚙️  No runtime cap configured (unlimited)

Benchmark separation by size category:
  L-size (highmem): 5 benchmarks, 107.8 hours (with 5 variants)
  S/M-size (standard): 281 benchmarks, 201.9 hours (with 5 variants)

TESTING L-SIZE BENCHMARKS (HIGHMEM MACHINES)

Testing 2 highmem VMs for L-size benchmarks:
Using 5 benchmarks with real runtime data (filtered from 5 total)
Target runtime per VM: 193950.8 seconds (53.9 hours)

=== L-size Balanced Partition (2 VMs) ===
Total VMs created: 2
Active VMs (with benchmarks): 2
Empty VMs: 0
Total runtime: 387901.6 seconds (107.8 hours)
Average runtime per active VM: 193950.8 seconds (53.9 hours)
Runtime standard deviation: 44160.9 seconds (12.3 hours)
Min runtime: 149789.9 seconds (41.6 hours)
Max runtime: 238111.7 seconds (66.

## Results Summary

In [59]:
# Print summary comparison table
print("\\n" + "="*80)
print("ALGORITHM COMPARISON SUMMARY")
print("="*80)

df_results = pd.DataFrame(results)

# Separate results by size category
l_results = df_results[df_results['size_category'] == 'L'] if 'size_category' in df_results.columns else pd.DataFrame()
sm_results = df_results[df_results['size_category'] == 'S/M'] if 'size_category' in df_results.columns else df_results

print(f"\\n{'Size':<6} {'VM Count':<9} {'Algorithm':<25} {'Efficiency':<12} {'Max Runtime (h)':<15} {'Std Dev (h)':<12}")
print("-" * 85)

for _, row in df_results.iterrows():
    size_cat = row.get('size_category', 'Mixed')
    alg_name = row['algorithm'].split('(')[0].strip()
    print(f"{size_cat:<6} {row['num_vms']:<9} {alg_name:<25} "
          f"{row['efficiency']:.3f}{'':8} {row['max_runtime']/3600:.1f}{'':12} "
          f"{row['std_runtime']/3600:.1f}")

# Find best configurations for each size category
print(f"\\n{'='*80}")
print("BEST CONFIGURATIONS:")
print(f"{'='*80}")

if len(l_results) > 0:
    best_l = l_results.loc[l_results['efficiency'].idxmax()]
    print(f"Best L-size (highmem): {best_l['num_vms']} VMs (efficiency: {best_l['efficiency']:.3f})")

if len(sm_results) > 0:
    best_sm = sm_results.loc[sm_results['efficiency'].idxmax()]
    print(f"Best S/M-size (standard): {best_sm['num_vms']} VMs (efficiency: {best_sm['efficiency']:.3f})")

# Calculate total deployment
if len(l_results) > 0 and len(sm_results) > 0:
    total_vms = best_l['num_vms'] + best_sm['num_vms']
    total_efficiency = (best_l['efficiency'] + best_sm['efficiency']) / 2
    print(f"\\nTotal deployment: {total_vms} VMs ({best_l['num_vms']} highmem + {best_sm['num_vms']} standard)")
    print(f"Average efficiency: {total_efficiency:.3f}")
elif len(sm_results) > 0:
    print(f"\\nTotal deployment: {best_sm['num_vms']} standard VMs only")
    print(f"Efficiency: {best_sm['efficiency']:.3f}")

\n================================================================================
ALGORITHM COMPARISON SUMMARY
\nSize   VM Count  Algorithm                 Efficiency   Max Runtime (h) Std Dev (h) 
-------------------------------------------------------------------------------------
L      2         L-size Balanced Partition 0.772         66.1             12.3
L      3         L-size Balanced Partition 0.630         47.4             13.3
L      4         L-size Balanced Partition 0.981         36.9             0.7
L      5         L-size Balanced Partition 0.741         36.9             7.0
S/M    8         S/M-size Balanced Partition 0.781         44.8             8.8
S/M    10        S/M-size Balanced Partition 0.862         35.8             4.6
S/M    12        S/M-size Balanced Partition 0.937         29.7             1.8
S/M    15        S/M-size Balanced Partition 0.807         24.1             4.3
\n===============================================================================

## Generate Optimal Allocation

In [60]:
# Generate optimal allocations for both size categories
print(f"\n\nGenerating optimal allocations with size-based machine separation...")

if MAX_RUNTIME_PER_VM_SECONDS is not None:
    print(f"Runtime cap: {MAX_RUNTIME_PER_VM_SECONDS}s ({MAX_RUNTIME_PER_VM_SECONDS/3600:.1f}h) per VM")

optimal_l_vms = []
optimal_sm_vms = []
best_l_result = None
best_sm_result = None

# Generate L-size allocation (highmem machines)
if len(l_results) > 0:
    best_l_result = l_results.loc[l_results['efficiency'].idxmax()]
    optimal_l_num_vms = best_l_result['num_vms']

    print(f"\nL-size benchmarks: {optimal_l_num_vms} highmem VMs")
    print(f"Efficiency: {best_l_result['efficiency']:.3f}")
    print(f"Max VM runtime: {best_l_result['max_runtime']/3600:.1f} hours")

    optimal_l_vms = balanced_partition(l_size_benchmarks, optimal_l_num_vms, MAX_RUNTIME_PER_VM_SECONDS)
    l_final_result = analyze_allocation(optimal_l_vms, f"Final L-size Allocation - Highmem")

# Generate S/M-size allocation (standard machines)
if len(sm_results) > 0:
    best_sm_result = sm_results.loc[sm_results['efficiency'].idxmax()]
    optimal_sm_num_vms = best_sm_result['num_vms']

    print(f"\nS/M-size benchmarks: {optimal_sm_num_vms} standard VMs")
    print(f"Efficiency: {best_sm_result['efficiency']:.3f}")
    print(f"Max VM runtime: {best_sm_result['max_runtime']/3600:.1f} hours")

    optimal_sm_vms = balanced_partition(non_l_benchmarks, optimal_sm_num_vms, MAX_RUNTIME_PER_VM_SECONDS)
    sm_final_result = analyze_allocation(optimal_sm_vms, f"Final S/M-size Allocation - Standard")

# Combined summary
total_vms = len(optimal_l_vms) + len(optimal_sm_vms)
total_runtime = sum(vm.total_runtime for vm in optimal_l_vms + optimal_sm_vms)

print(f"\n{'='*60}")
print("FINAL ALLOCATION SUMMARY")
print(f"{'='*60}")
print(f"Total VMs: {total_vms}")
print(f"  - Highmem VMs (L-size): {len(optimal_l_vms)}")
print(f"  - Standard VMs (S/M-size): {len(optimal_sm_vms)}")
print(f"Total allocated runtime: {total_runtime/3600:.1f} hours")
print(f"Machine separation ensures optimal resource utilization")



Generating optimal allocations with size-based machine separation...

L-size benchmarks: 4 highmem VMs
Efficiency: 0.981
Max VM runtime: 36.9 hours
Using 5 benchmarks with real runtime data (filtered from 5 total)
Target runtime per VM: 96975.4 seconds (26.9 hours)

=== Final L-size Allocation - Highmem ===
Total VMs created: 4
Active VMs (with benchmarks): 3
Empty VMs: 1
Total runtime: 387901.6 seconds (107.8 hours)
Average runtime per active VM: 129300.5 seconds (35.9 hours)
Runtime standard deviation: 2468.7 seconds (0.7 hours)
Min runtime: 127407.5 seconds (35.4 hours)
Max runtime: 132787.5 seconds (36.9 hours)
Runtime ratio (max/min): 1.04
Load balance efficiency: 0.981 (1.0 = perfect balance)

S/M-size benchmarks: 12 standard VMs
Efficiency: 0.937
Max VM runtime: 29.7 hours
Using 281 benchmarks with real runtime data (filtered from 281 total)
Target runtime per VM: 60559.1 seconds (16.8 hours)

=== Final S/M-size Allocation - Standard ===
Total VMs created: 12
Active VMs (with 

## Export Configuration

In [61]:
# Export the allocation to YAML files for infrastructure
# NOTE: This exports ONLY benchmarks with real runtime data, separated by size category
# ONLY exports VMs that have benchmarks assigned (skips empty VMs)
output_dir = Path('../infrastructure/benchmarks/runtime_optimized')
output_dir.mkdir(exist_ok=True, parents=True)

# Clear existing files
for file in output_dir.glob('*.yaml'):
    file.unlink()

exported_vms = 0
total_benchmarks_exported = 0

# Filter to only VMs with benchmarks
active_l_vms = [vm for vm in optimal_l_vms if vm.benchmarks]
active_sm_vms = [vm for vm in optimal_sm_vms if vm.benchmarks]

print(f"Exporting {len(active_l_vms)} highmem VMs and {len(active_sm_vms)} standard VMs (skipping empty VMs)\n")

# Export L-size VMs (highmem machines)
for vm_idx, vm in enumerate(active_l_vms):
    # L-size benchmarks always get highmem machines
    machine_type = 'c4-highmem-8'
    years = [2025]  # Include highs-hipo for L benchmarks

    # Create benchmark structure with runtime metadata
    benchmarks_dict = {}
    for benchmark in vm.benchmarks:
        benchmark_name = benchmark['name']
        if benchmark_name not in benchmarks_dict:
            benchmarks_dict[benchmark_name] = {'Sizes': []}

        size_entry = {
            'Name': benchmark['size_name'],
            'Size': benchmark['size_category'],
            'URL': benchmark['url'],
            '_runtime_s': round(benchmark['runtime'], 2)  # Add runtime for cross-checking
        }
        benchmarks_dict[benchmark_name]['Sizes'].append(size_entry)

    # Create YAML content with total runtime metadata
    yaml_content = {
        'machine-type': machine_type,
        'years': years,
        '_total_runtime_s': round(vm.total_runtime, 2),  # Total runtime for this VM
        '_total_runtime_h': round(vm.total_runtime / 3600, 2),  # In hours for readability
        '_num_benchmarks': len(vm.benchmarks),
        'benchmarks': benchmarks_dict
    }

    # Write to file
    filename = f'highmem_vm_{vm_idx:02d}.yaml'
    with open(output_dir / filename, 'w') as f:
        yaml.safe_dump(yaml_content, f, default_flow_style=False, sort_keys=False)

    print(f"Exported {filename}: {len(vm.benchmarks)} L-size benchmarks, "
          f"{vm.total_runtime/3600:.1f}h runtime")

    total_benchmarks_exported += len(vm.benchmarks)

# Export S/M-size VMs (standard machines)
for vm_idx, vm in enumerate(active_sm_vms):
    # S/M-size benchmarks get standard machines
    machine_type = 'c4-standard-2'
    # years = [2020, 2022, 2023, 2024, 2025]
    years = [2025]

    # Create benchmark structure with runtime metadata
    benchmarks_dict = {}
    for benchmark in vm.benchmarks:
        benchmark_name = benchmark['name']
        if benchmark_name not in benchmarks_dict:
            benchmarks_dict[benchmark_name] = {'Sizes': []}

        size_entry = {
            'Name': benchmark['size_name'],
            'Size': benchmark['size_category'],
            'URL': benchmark['url'],
            '_runtime_s': round(benchmark['runtime'], 2)  # Add runtime for cross-checking
        }
        benchmarks_dict[benchmark_name]['Sizes'].append(size_entry)

    # Create YAML content with total runtime metadata
    yaml_content = {
        'machine-type': machine_type,
        'years': years,
        '_total_runtime_s': round(vm.total_runtime, 2),  # Total runtime for this VM
        '_total_runtime_h': round(vm.total_runtime / 3600, 2),  # In hours for readability
        '_num_benchmarks': len(vm.benchmarks),
        'benchmarks': benchmarks_dict
    }

    # Write to file
    filename = f'standard_{vm_idx:02d}.yaml'
    with open(output_dir / filename, 'w') as f:
        yaml.safe_dump(yaml_content, f, default_flow_style=False, sort_keys=False)

    print(f"Exported {filename}: {len(vm.benchmarks)} S/M-size benchmarks, "
          f"{vm.total_runtime/3600:.1f}h runtime")

    total_benchmarks_exported += len(vm.benchmarks)

total_exported_vms = len(active_l_vms) + len(active_sm_vms)

print(f"\n{'='*70}")
print(f"Configuration files written to {output_dir}/")
print(f"Total VMs exported: {total_exported_vms} (skipped {len(optimal_l_vms) + len(optimal_sm_vms) - total_exported_vms} empty VMs)")
print(f"  - Highmem VMs: {len(active_l_vms)}")
print(f"  - Standard VMs: {len(active_sm_vms)}")
print(f"Total benchmarks exported: {total_benchmarks_exported}")
print(f"Total runtime allocated: {sum(vm.total_runtime for vm in active_l_vms + active_sm_vms)/3600:.1f} hours")
print(f"\nMACHINE SEPARATION POLICY:")
print(f"  - L-size benchmarks → c4-highmem-8 (high memory for large problems)")
print(f"  - S/M-size benchmarks → c4-standard-2 (cost-effective for smaller problems)")
print(f"\nNOTE: Only benchmarks with real HiGHS runtime data were included.")

Exporting 3 highmem VMs and 7 standard VMs (skipping empty VMs)

Exported highmem_vm_00.yaml: 1 L-size benchmarks, 36.9h runtime
Exported highmem_vm_01.yaml: 2 L-size benchmarks, 35.5h runtime
Exported highmem_vm_02.yaml: 2 L-size benchmarks, 35.4h runtime
Exported standard_00.yaml: 9 S/M-size benchmarks, 29.6h runtime
Exported standard_01.yaml: 10 S/M-size benchmarks, 29.7h runtime
Exported standard_02.yaml: 10 S/M-size benchmarks, 29.3h runtime
Exported standard_03.yaml: 12 S/M-size benchmarks, 29.6h runtime
Exported standard_04.yaml: 13 S/M-size benchmarks, 29.6h runtime
Exported standard_05.yaml: 23 S/M-size benchmarks, 29.7h runtime
Exported standard_06.yaml: 204 S/M-size benchmarks, 24.4h runtime

Configuration files written to ../infrastructure/benchmarks/runtime_optimized/
Total VMs exported: 10 (skipped 6 empty VMs)
  - Highmem VMs: 3
  - Standard VMs: 7
Total benchmarks exported: 286
Total runtime allocated: 309.6 hours

MACHINE SEPARATION POLICY:
  - L-size benchmarks → c4-h